In [4]:
import pandas as pd
import re
import random
import sys
import requests
import json 
import argparse
import math 
import os
import graph 
import sys 
import json 
import pickle
import subprocess

In [5]:
df = pd.read_csv("city_bounds_US urban_rural_mod.csv",index_col=0)

df=pd.concat([df,df["Lat-Long 1"].str.split(",",expand=True),df["Lat-Long 2"].str.split(",",expand=True)],axis=1)
df.columns=list(df.columns[:-4])+["Lat1","Long1","Lat2","Long2"]

In [6]:
df["output_name"]="output/"+df["City"].apply(lambda x: re.sub('[^a-zA-Z]', "", x))+".json"
df["Lat1"]=df["Lat1"].apply(lambda x: float(re.sub(r'[^\d.-]+', "", x)))
df["Long1"]=df["Long1"].apply(lambda x: float(re.sub(r'[^\d.-]+', "", x)))
df["Lat2"]=df["Lat2"].apply(lambda x: float(re.sub(r'[^\d.-]+', "", x)))
df["Long2"]=df["Long2"].apply(lambda x: float(re.sub(r'[^\d.-]+', "", x)))
df["New City"]=df["City"].apply(lambda x: re.sub('[^a-zA-Z]', "", x))

In [7]:
df["Samp_Lat"]=round(sum([df["Lat1"],df["Lat2"]])/2,6)
df["Samp_Long"]=round(sum([df["Long1"],df["Long2"]])/2,6)

In [5]:
def infer_mapbox_input(lat,lon,tile_size,model_id,output,osm_only=0):
    #args = parseArgument()
    print(lat,lon,tile_size,model_id,output,osm_only)
    msg = {}
    msg["lat"] = lat
    msg["lon"] = lon
    msg["v_thr"] = 0.05
    msg["e_thr"] = 0.01
    msg["snap_dist"] = 15
    msg["snap_w"] = 100
    msg["model_id"] = model_id

    msg["size"] = tile_size;
    n = int(math.ceil(tile_size / 176.0))
    msg["padding"] = (n * 176 - tile_size)//2;
    msg["stride"] = 176;
    msg["nPhase"] = 1;
    if model_id == 3:
        msg["nPhase"] = 5;
    
    if osm_only != 0:
        msg["osm"] = True

    url = "http://localhost:8011"
    print("debug1")
    try:
        x = requests.post(url, data = json.dumps(msg),timeout=300)
    except requests.exceptions.ReadTimeout:
        print("TIMEOUT: ",output)
        return
    print("debug2")
    graph = json.loads(x.text) 
    if graph["success"] == 'false':
        print("unknown error")
        exit()

    #print(graph)
    print("debug3")
    if osm_only != 0:
        json.dump(graph["osmgraph"], open(output, "w"), indent=2)
    else:
        json.dump(graph["graph"]["graph"][0], open(output, "w"), indent=2)
    
    tid = graph["taskid"]
    print("please check intermediate results at http://localhost:8010/t%d/" % tid)
    return

In [6]:
def get_prop_gt(df,model_id=2,tile_size=500):
    for i in range(df.shape[0]):
        infer_mapbox_input(float(df.iloc[i]["Samp_Lat"]),float(df.iloc[i]["Samp_Long"]),tile_size,model_id,df.iloc[i]["output_name"])
        infer_mapbox_input(float(df.iloc[i]["Samp_Lat"]),float(df.iloc[i]["Samp_Long"]),tile_size,model_id,"gt_"+df.iloc[i]["output_name"],osm_only=1)

In [7]:
def pickler(infolder,outfolder):
    for file in os.listdir(infolder):
        infile=infolder+file
        outfile=outfolder+file[:-5]+".p"
        
        jsonedges = json.load(open(infile))
        picklegraph = graph.link2graph(jsonedges)

        pickle.dump(picklegraph, open(outfile, "wb"))


In [8]:
def json_convert(infolder,outfolder):
    lat_top_left = 41.0 
    lon_top_left = -71.0 
    min_lat = 41.0 
    max_lon = -71.0 

    def xy2latlon(x,y):
            lat = lat_top_left - x * 1.0 / 111111.0
            lon = lon_top_left + (y * 1.0 / 111111.0) / math.cos(math.radians(lat_top_left))

            return lat, lon 

    for file in os.listdir(infolder):
        f_in = infolder+file
        f_out = outfolder+file[:-2]+".json"


        try:
                neighbors = pickle.load(open(f_in, "r"))
        except:
                neighbors = pickle.load(open(f_in, "rb"),encoding="bytes")


        nodes = []
        edges = []

        cc=0

        nodemap = {}
        edge_map = {}

        for k, v in neighbors.items():
                nodemap[k] = len(nodes)

                n1 = k 
                lat1,lon1 = xy2latlon(n1[0], n1[1])

                nodes.append([lat1,lon1])


        for k, v in neighbors.items():
                n1 = k 

                for n2 in v:
                    if (n1,n2) in edge_map or (n2,n1) in edge_map:
                        continue
                    else:
                        edge_map[(n1,n2)] = True 


                    edges.append([nodemap[n1], nodemap[n2]])


        json.dump([nodes,edges], open(f_out, "w"), indent=2)





In [9]:
def check_apls(gtfolder,propfolder,outfolder):
    for file in os.listdir(propfolder):
        subprocess.run(["go","run","apls/main.go",gtfolder+file,propfolder+file,outfolder+file[:-5]+".txt"])

In [21]:
def get_apls_results(folder):
    res={"New City":[],"recall":[],"precision":[]}
    for file in os.listdir(folder):
        f = open(folder+file, "r")
        scores=[float(x) for x in f.read().split(" ")]
        f.close()
        res["New City"].append(file[:-4])
        res["recall"].append(scores[0])
        res["precision"].append(scores[1])
    return pd.DataFrame.from_dict(res)

In [13]:
get_prop_gt(df,model_id=5)
pickler("output//","prop_pickles//")
pickler("gt_output//","gt_pickles//")
json_convert("prop_pickles//","prop_jsons//")
json_convert("gt_pickles//","gt_jsons//")
check_apls("gt_jsons//","prop_jsons//","apls_results//")
#get_apls_results("apls_results//")

,New City,Model5_recall,Model5_precision
0,Athens,0.027557,0.027557
1,Birmingham,0.251069,0.251069
2,Columbus,0.354161,0.354161
3,Concord,0.218240,0.218240
4,Cornelius,0.197043,0.197043
5,CountryWalk,0.030318,0.030318
6,Derby,0.629575,0.629575
7,Guttenberg,0.472015,0.472015
8,LittleRock,0.103108,0.103108
9,LosAngeles,0.085716,0.085716


In [26]:
mod2_table=pd.merge(df,get_apls_results("Model2 Files//apls_results//"))

In [27]:
mod2_table.to_csv("mod2_results.csv")